In [1]:
PILE_DIR="nl-en/"
import load_dataset
from transformers import GPT2Tokenizer
import csv
import json
import os
from absl import logging
import IPython
import sys
logging.get_absl_handler().python_handler.stream = sys.stdout



In [17]:
# some code to count the max number of tokens in a sentence in the dataset

import nltk
nltk.download('punkt')

def max_tokens_in_sentence(file_path):
    max_tokens = 0

    with open(file_path, 'r') as file:
        i = 0
        for line in file:
            i +=1
            if(i%1000) == 0:
                print(i)
            if(line):
            # Tokenize the sentence
                tokens = nltk.word_tokenize(line)
                num_tokens = len(tokens)
                # Update max_tokens if current sentence has more tokens
                if num_tokens > max_tokens:
                    max_tokens = num_tokens
    
    return max_tokens

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/annavisman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
path = "/Users/annavisman/stack/RUG/CS/Year3/thesis/thesis-llm-privacy/nl-en/europarl-v7.nl-en.en"
#max_tokens = max_tokens_in_sentence(path)
#print("Maximum number of tokens in a sentence:", max_tokens)

NameError: name 'max_tokens_in_sentence' is not defined

In [51]:
# Function to tokenize a sentence and return its length
def tokenize_sentence(sentence, tokenizer):
    tokens = tokenizer.encode(sentence, max_length=1024, truncation=True)
    return len(tokens)

# Function to generate the csv byte offset file from the original dataset
def generate_dataset(input_file, output_file, tokenizer):
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    with open(output_file, "w", newline='', encoding="utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["exid", "fid", "line_byte_offset", "start", "end", "take_before", "take_after", "internal_offset", "size", "start_byte", "end_byte", "count"])
        
        line_byte_offset = 0
        exid = 0
        fid = 0
        for line in lines:
            # Remove leading/trailing whitespaces and newline characters
            line = line.strip()
            
            # Calculate the end position (end of sentence)
            end = len(line) - 1

            # Tokenize the sentence and get its length
            size = tokenize_sentence(line, tokenizer)
            
            # Write the row to the CSV file
            csv_writer.writerow([exid, fid, line_byte_offset, 0, end, 0, 0, 0, size, -1, -1, -1])
            
            # Update line byte offset for the next sentence
            line_byte_offset += len(line) + 1  # Add 1 for the newline character
            exid += 1  # Increment example ID for the next sentence

In [52]:
path = "/Users/annavisman/stack/RUG/CS/Year3/thesis/thesis-llm-privacy/nl-en/europarl-v7.nl-en-200.en"

output_file = "datasets/train_dataset_en_200.csv"
# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Generate the dataset
generate_dataset(path, output_file, tokenizer)
    
print("Dataset generated successfully!")

Dataset generated successfully!


In [42]:
def truncate_sentence(sentence, max_tokens):
    # Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)
    
    # Truncate to max_tokens tokens
    truncated_tokens = tokens[:max_tokens]
    
    # Convert tokens back to string
    truncated_sentence = tokenizer.convert_tokens_to_string(truncated_tokens)
    
    return truncated_sentence

def filter_and_truncate_sentences(input_file, output_file, max_tokens):
    with open(input_file, "r", encoding="utf-8") as f_input, \
         open(output_file, "w", encoding="utf-8") as f_output:
        
        for line in f_input:
            # Remove leading/trailing whitespaces and newline characters
            sentence = line.strip()
            
            # Tokenize the sentence
            tokens = tokenizer.tokenize(sentence)
            
            # Check if the number of tokens exceeds the maximum
            if len(tokens) >= max_tokens:
                # Truncate the sentence to max_tokens tokens
                truncated_sentence = truncate_sentence(sentence, max_tokens)
                
                # Write the truncated sentence to the output file
                f_output.write(truncated_sentence + "\n")


In [44]:
# stripping the original dataset to sentences of 200 tokens
input = "nl-en/europarl-v7.nl-en.en"

output = "nl-en/europarl-v7.nl-en-100.en"

# Filter and truncate lines in the input file
filter_and_truncate_sentences(input, output, 100)

print("Filtered and truncated lines written to:", output)

Filtered and truncated lines written to: nl-en/europarl-v7.nl-en-100.en


In [7]:
# i want to know how many entries have more than 200 tokens in a sentence

def count_large_entries(csv_file, tokens):
    # Open the CSV file for reading
    with open(csv_file, "r", newline='', encoding="utf-8") as csvfile:
        csv_reader = csv.DictReader(csvfile)
        
        # Initialize a counter for large entries
        large_entry_count = 0
        
        # Iterate through each row in the CSV file
        for row in csv_reader:
            # Convert the value in the "size" column to an integer
            # this is the number of tokens in the example
            size = int(row["size"])
            
            # Check if the size is greater than or equal to the amount of tokens supplied
            if size >= tokens:
                # Increment the counter if the condition is met
                large_entry_count += 1
                
    return large_entry_count

In [8]:
csv_file = "datasets/train_dataset_en.csv"
tokens = 200

# Count the number of large entries
count = count_large_entries(csv_file, tokens)
    
print("Number of rows with size >= ", tokens, ":", count)

NameError: name 'csv' is not defined

In [48]:
# generating a jsonlines version of dataset to work with llm benchmark code

def text_to_jsonlines(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f_input, \
         open(output_file, "w", encoding="utf-8") as f_output:
        
        for line in f_input:
            # Remove leading/trailing whitespaces and newline characters
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
            
            # Create a JSON object with a "text" field containing the line
            json_object = {"text": line}
            
            # Write the JSON object to the output file as a single line
            json.dump(json_object, f_output)
            f_output.write('\n')

In [49]:
input_file = "nl-en/europarl-v7.nl-en-200.en"
output_file = "nl-en/europarl-v7.nl-en-200.en.jsonl"

# Convert the plain text dataset to JSON Lines format
text_to_jsonlines(input_file, output_file)
    
print("Conversion completed successfully!")

Conversion completed successfully!


In [67]:
# running llm benchmark dataset split code to generate prefixes and suffixes

!python load_dataset.py nl-en/ train en

Opened file nl-en/europarl-v7.nl-en-200.en.jsonl
Generating prefix and suffix for split train in language en
Headers:
['exid', 'fid', 'line_byte_offset', 'start', 'end', 'take_before', 'take_after', 'internal_offset', 'size', 'start_byte', 'end_byte', 'count']
MD5 hash of dataset:
5eacff84e8b7ebdc4a364573806ddfb3
Done!


In [67]:
import numpy as np

# Specify the file path of the .npy file
file_path = "datasets/train_dataset.npy"

# Load the .npy file
data = np.load(file_path)

# View the first 10 entries
print(data[:10])

print("done")

[[  464 34754   254 ... 34754   254     7]
 [  464 34754   254 ... 34754   254     7]
 [  464 34754   254 ...   254  1640 34754]
 ...
 [  464 34754   254 ...   254  1659 34754]
 [ 3152 34754   254 ...   592 34754   254]
 [  818 34754   254 ...  1169 34754   254]]
done
